In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/Henon_non_chaotic_T10_20_25_30_35_40_45_50/Henon_non_chaotic_T10_45'
def generate_random_values():
    a1 = -0.5
    a2 = 0.5
    a3 = -0.1
    a4 = 0.1
    x1 = np.random.uniform(a1, a2)
    x2 = np.random.uniform(a1, a2)
    x3 = np.random.uniform(a3, a4)
    x4 = np.random.uniform(a3, a4)
    return x1, x2, x3, x4
def generate_random_values_based_on_c():
    a1 = -0.5
    a2 = 0.5
    a3 = -0.1
    a4 = 0.1
    x1 = np.random.uniform(a1, a2)
    x2 = np.random.uniform(a1, a2)
    x3 = np.random.uniform(a3, a4)
    x4 = np.random.uniform(a3, a4)
    return x1, x2, x3, x4
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4 = y
        f = np.array([x3, x4, -x1 - 2*x1*x2, -x2 - x1**2 + x2**2])
        normalized_f = normalize(f)
        return normalized_f
    def compute_energy(x1, x2, x3, x4):
        return 0.5 * (x3**2 + x4**2) + 0.5 * (x1**2 + x2**2) + x1**2 * x2 - (1/3) * x2**3
    num_trajectories = 10
    t = np.linspace(0, 10, 45) # 40 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
        final_state = sol[-1, :]
        E = compute_energy(*final_state)
        initial_conditions_to_print.append((initial_condition, E))
    num_variables = 4 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row) 
    for r, (initial_conditions, E) in enumerate(initial_conditions_to_print):
        print(f"Energy for Trajectory {r+1}: {E}")
    output_directory1 = r'../../../results/Henon_non_chaotic_T10_20_25_30_35_40_45_50/Henon_non_chaotic_T10_45'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('../../../Data/Henon_non_chaotic_T10_20_25_30_35_40_45_50/Henon_non_chaotic_T10_45/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4 = generate_random_values()
    initial_conditions = [generate_random_values_based_on_c() for _ in range(10)]  # number of trajectories
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 11):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4):
(-0.1876864961412784, 0.29016668554434855, 0.008728927925464716, 0.05932960427061973)
(0.38993339049592235, 0.052870965799064806, 0.052221773565617635, 0.058296553816782504)
(0.2082949951261468, -0.10700668142426084, -0.09982708262460567, 0.0219807167294475)
(-0.1142232957846051, -0.328077403174613, 0.09764728396230399, -0.05390323078916051)
(0.32271614079138167, -0.22978886216905992, -0.06968542952516939, 0.029692037712074237)
(0.2298211276366834, 0.43039062181683796, -0.07517061702871819, 0.013387722529096771)
(-0.1359815466297689, 0.48757741553140566, -0.031654056903026515, 0.027818429636957742)
(0.20508170874294263, -0.25694754530435715, 0.041218038782937755, -0.07634029578740248)
(-0.24878631825464026, -0.21759851450480627, -0.039406750165312365, 0.007894945533285777)
(-0.32021661150614766, -0.48629933270620085, 0.05408875819563247, 0.07958857860164106)
Energy for Trajectory 1: 0.06358734842564608
Energy for Trajectory 2: 0.08847414366650477
Energy f

In [1]:
import pandas as pd
import os
input_directory = r'../../../System/Henon_non_chaotic_T10_20_25_30_35_40_45_50/Henon_non_chaotic_T10_45'
output_directory = r'../../../Data/Henon_non_chaotic_T10_20_25_30_35_40_45_50/Henon_non_chaotic_T10_45/Traj_data'
df14 = pd.read_csv(os.path.join(input_directory, '50.csv')) # 50.csv, saved names of the data
Br = pd.read_csv('../../../System/Henon_non_chaotic_T10_20_25_30_35_40_45_50/Henon_non_chaotic_T10_45/50.csv')
tr = Br.groupby('trajectory').size()
re1 = int(round(tr.mean()))
rows_per_file = re1 
num_files = len(df14) // rows_per_file
data_chunks = [df14.iloc[i * rows_per_file:(i + 1) * rows_per_file].iloc[:, :-1] for i in range(num_files)]
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f'tr_{i + 1}.csv'), index=False)